In [ ]:
# === 1. Import Libraries ===
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Layer
from sklearn.preprocessing import MinMaxScaler

# === 2. Define Custom Layer ===
@tf.keras.utils.register_keras_serializable()
class DayNightPenalty(Layer):
    def __init__(self, vtec_night_limit=10.0, vtec_day_min=30.0, penalty_scale=0.1, **kwargs):
        super(DayNightPenalty, self).__init__(**kwargs)
        self.vtec_night_limit = vtec_night_limit
        self.vtec_day_min = vtec_day_min
        self.penalty_scale = penalty_scale

    def call(self, inputs, **kwargs):
        vtec_pred, hour_val = inputs
        is_night = tf.cast(tf.logical_or(hour_val < 6, hour_val > 18), tf.float32)
        is_day = 1.0 - is_night
        night_excess = tf.nn.relu(vtec_pred - self.vtec_night_limit)
        day_deficit = tf.nn.relu(self.vtec_day_min - vtec_pred)
        penalty = self.penalty_scale * (is_night * night_excess + is_day * day_deficit)
        self.add_loss(tf.reduce_mean(penalty))
        return vtec_pred

# === 3. Load Dataset ===
data_path = "D:\\Graduation Project V3\\final north data.xlsx"
model_path = "D:\\Graduation Project V3\\LSTM\\final_trained_lstm_model.h5"

df = pd.read_excel(data_path)
df["datetime"] = pd.to_datetime(df["datetime"])
df["date_str"] = df["datetime"].dt.strftime("%Y-%m-%d")
df["Hour"] = df["datetime"].dt.hour
df["DayOfYear"] = df["datetime"].dt.dayofyear

# # === 4. Feature Engineering ===
# df["HoD_sin"] = np.sin(2 * np.pi * df["Hour"] / 24)
# df["HoD_cos"] = np.cos(2 * np.pi * df["Hour"] / 24)
# df["DoY_sin"] = np.sin(2 * np.pi * df["DayOfYear"] / 365.25)
# df["DoY_cos"] = np.cos(2 * np.pi * df["DayOfYear"] / 365.25)
# df["VTEC_EMA_4day"] = df["Average_TEC"].ewm(span=96, adjust=False).mean()
# df["VTEC_EMA_30day"] = df["Average_TEC"].ewm(span=720, adjust=False).mean()
# df["VTEC_derivative_1st"] = df["Average_TEC"].diff(1)
# df["VTEC_derivative_2nd"] = df["VTEC_derivative_1st"].diff(1)
# df["VTEC_lag_1"] = df["Average_TEC"].shift(1)
# df["VTEC_lag_24"] = df["Average_TEC"].shift(24)
# df.dropna(inplace=True)

# === 5. Feature List and Target ===
features = [
    "Average_Lat", "Average_Long", "Average_BS", "Average_BZ", "Average_SWD",
    "Average_SWS", "Average_EF", "Average_Kp", "Average_Rs", "Average_Dst",
    "Average_f10", "Average_MagMN", "VTEC_EMA_4day", "VTEC_EMA_30day",
    "VTEC_derivative_1st", "VTEC_derivative_2nd", "VTEC_lag_1", "VTEC_lag_24",
    "HoD_sin", "HoD_cos", "DoY_sin", "DoY_cos"
]
target = "Average_TEC"

# === 6. Scale Features ===
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

df[features] = scaler_X.fit_transform(df[features])
df[target] = scaler_y.fit_transform(df[[target]])

# === 7. Load Model with Custom Layer ===
model = load_model(model_path, custom_objects={"DayNightPenalty": DayNightPenalty})

# === 8. Forecast Function ===
def forecast_day(df_original, date_str, lookback_window=72):
    df_day = df_original[df_original["date_str"] == date_str].copy()
    if df_day.empty:
        print(f"❌ No data for {date_str}")
        return

    df_day = df_day.reset_index(drop=True)
    base_idx = df_original[df_original["datetime"] == df_day.iloc[0]["datetime"]].index
    if base_idx.empty:
        print(f"❌ Base datetime not found")
        return
    base_idx = base_idx[0]

    X_final, T_final = [], []
    for i in range(len(df_day)):
        if base_idx + i - lookback_window < 0:
            continue
        seq_slice = df_original.iloc[base_idx + i - lookback_window: base_idx + i]
        if len(seq_slice) != lookback_window:
            continue
        X_seq = seq_slice[features].values
        hour_val = df_day.iloc[i]["Hour"]
        X_final.append(X_seq)
        T_final.append(hour_val)

    if not X_final:
        print(f"❌ Not enough data to forecast {date_str}")
        return

    X_final = np.array(X_final)
    T_final = np.array(T_final).reshape(-1, 1)

    pred_scaled = model.predict([X_final, T_final], verbose=0)
    pred = scaler_y.inverse_transform(pred_scaled)
    true = scaler_y.inverse_transform(df_day[target].values[:len(pred)].reshape(-1, 1))

    # === Plot ===
    plt.figure(figsize=(10, 4))
    plt.plot(range(len(true)), true, label="Actual VTEC", marker='o')
    plt.plot(range(len(pred)), pred, label="Predicted VTEC", marker='s')
    plt.title(f"Forecast on {date_str}")
    plt.xlabel("Time Step")
    plt.ylabel("VTEC")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

# === 9. Run Forecast ===
forecast_day(df, "2024-07-28")


In [ ]:
forecast_day(df, "2022-04-28")

In [ ]:
forecast_day(df, "2022-04-01")

In [ ]:
forecast_day(df, "2023-04-09")

# REMOVE ERROR BARS

In [ ]:
# REMOVE ERROR BARS
def forecast_day(df_original, date_str, lookback_window=72):
    df_day = df_original[df_original["date_str"] == date_str].copy()
    if df_day.empty:
        print(f"❌ No data for {date_str}")
        return

    df_day = df_day.reset_index(drop=True)
    base_idx = df_original[df_original["datetime"] == df_day.iloc[0]["datetime"]].index
    if base_idx.empty:
        print(f"❌ Base datetime not found")
        return
    base_idx = base_idx[0]

    X_final, T_final = [], []
    for i in range(len(df_day)):
        if base_idx + i - lookback_window < 0:
            continue
        seq_slice = df_original.iloc[base_idx + i - lookback_window: base_idx + i]
        if len(seq_slice) != lookback_window:
            continue
        X_seq = seq_slice[features].values
        hour_val = df_day.iloc[i]["Hour"]
        X_final.append(X_seq)
        T_final.append(hour_val)

    if not X_final:
        print(f"❌ Not enough data to forecast {date_str}")
        return

    X_final = np.array(X_final)
    T_final = np.array(T_final).reshape(-1, 1)

    pred_scaled = model.predict([X_final, T_final], verbose=0)
    pred = scaler_y.inverse_transform(pred_scaled)
    true = scaler_y.inverse_transform(df_day[target].values[:len(pred)].reshape(-1, 1))

    # === Plot ===
    plt.figure(figsize=(10, 4))
    plt.plot(range(len(true)), true, label="Actual VTEC", linewidth=2)  # no marker
    plt.plot(range(len(pred)), pred, label="Predicted VTEC", linewidth=2)  # no marker
    plt.title(f"Forecast on {date_str}")
    plt.xlabel("Time Step")
    plt.ylabel("VTEC")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

forecast_day(df, "2023-04-09")


In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"Matplotlib version: {matplotlib.__version__}")
print(f"TensorFlow version: {tf.__version__}")
print(f"Scikit-learn version: {MinMaxScaler.__module__.split('.')[0]} {pd.__version__}")